In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import json
import requests
import string
import re   # regex
import nltk # text handling
from nltk.stem import WordNetLemmatizer
from nltk.stem.isri import ISRIStemmer
from textblob import TextBlob # WordCloud

In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas
import seaborn as sns 
from sklearn import preprocessing
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# To see the whole tweet text
pd.set_option('display.max_colwidth', None)

# Data Exploration

In [5]:
# load specific columns from the csv file
#col_list = ["created_at", "id", "in_reply_to_status_id", 
 #           "in_reply_to_user_id", "text", "user_screen_name"]

tweets = pd.read_excel('Sentiment Analysis Training.xlsx')
#
 #                 , usecols = col_list)


In [6]:
len(tweets)

133

In [7]:
len(tweets) 

133

# Text Preparation

In [8]:
mentions = re.compile('(@[a-zA-Z0-9_]{1,50})') # mentions regex @...
urls = re.compile('http\S+')         # links regex
diacritics = re.compile('ٰ ّ َ ً ُ ٌ ِ ٍ ْـ')       # Tashkeel regex

emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

arabic_punctuations = '''`÷×؛<>()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ«»'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

stopwords = nltk.corpus.stopwords.words('arabic')

def remove_mentions(text):
    text = re.sub(mentions, '', text)
    return text

# removing URLs
def remove_urls(text):
    # twitter converts all links to thier own domain t.co
    text = re.sub(urls, '', text)
    return text

# removing tashkeel
def remove_diacritics(text):
    text = re.sub(r'[\u064b-\u065f]', '', text)
    return text

def remove_emojis(text):
    # https://www.linkedin.com/pulse/extracting-twitter-data-pre-processing-sentiment-using-jayasekara
    return re.sub(emoji_pattern, '', text)

def remove_repeating_char(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    return re.sub(r'(.)\1+', r'\1', text)

def lemmatize_text(text):
    # lemmatizes text
    lemmatized_text = []

    # set up arabic lemmatizer Farasa
    url = 'https://farasa.qcri.org/webapi/lemmatization/'
    api_key = "lErIOPgmHZtflLMgIf"

    # set up english lemmatizer
    eng_lemmatizer = nltk.stem.WordNetLemmatizer()

    for word in text:
        # Detect language to use proper lemmatizer
        if TextBlob(word).detect_language() == 'en':
            lemmatized_text.append(eng_lemmatizer.lemmatize(word))
        else:
            payload = {'text': word, 'api_key': api_key}
            data = requests.post(url, data=payload)
            lemmatized_text.append(json.loads(data.text))
    return lemmatized_text

def text_stemming(text):
    return ISRIStemmer().suf32(text)

def remove_punctuations(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    translator = str.maketrans(' ', ' ', punctuations_list)
    return text.translate(translator)

def normalize_arabic(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
    
# remove hashtags marks but keep the words itself
def normalize_hashtags(text):
    text = re.sub("#", "", text)
    text = re.sub("_", " ", text)
    text = re.sub("_", " ", text)
    return text

In [9]:
normalized_stopwords = map(normalize_arabic, stopwords)
normalized_stopwords = list(normalized_stopwords)
normalized_stopwords = list(set(normalized_stopwords))
normalized_stopwords[:5]

['اض', 'بما', 'راح', 'امام', 'ترك']

In [10]:
# clean text
def text_preprocessing(text):
    text = remove_diacritics(text)
    text = remove_mentions(text)
    text = normalize_hashtags(text)
    text = remove_urls(text)
    text = remove_emojis(text)
    text = remove_repeating_char(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = ' '.join(word for word in text.split() if word not in normalized_stopwords)
    return text.split()

In [11]:
tweets['text']=tweets['text'].apply(str)

In [12]:
tweets['clean_tweet'] = tweets['text'].apply(text_preprocessing)

# Vectorization

## Word2Vec

In [13]:
file_model = tweets.copy()
file_model = file_model[file_model.clean_tweet.str.len()>1]

In [14]:
file_model['clean_tweet'][0]

['ابشرك',
 'بتدخل',
 'اليوم',
 'العاشر',
 'عشان',
 'يردوا',
 'يخدمونك',
 'ياخي',
 'تحس',
 'stc',
 'مايدرون',
 'رءيه',
 '٢٠٣٠',
 'الحين',
 'ونفس',
 'الاخطاء',
 'عشرات',
 'السنين']

In [15]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from time import time 
import multiprocessing

In [16]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [17]:
# combine common expressions from 2 words that are repeated in the data
sent = [row for row in file_model.clean_tweet]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1] # print the first tweet after the embedding

INFO - 05:37:57: collecting all words and their counts
INFO - 05:37:57: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 05:37:57: collected 1839 word types from a corpus of 1239 words (unigram + bigrams) and 125 sentences
INFO - 05:37:57: using 1839 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 05:37:57: source_vocab length 1839
INFO - 05:37:57: Phraser built with 55 phrasegrams


['ابشرك', '٣خطوط', 'نقلتها', 'وتوبه', 'وماعاد', 'رجعه']

In [18]:
file_model['clean_tweet']

0      [ابشرك, بتدخل, اليوم, العاشر, عشان, يردوا, يخدمونك, ياخي, تحس, stc, مايدرون, رءيه, ٢٠٣٠, الحين, ونفس, الاخطاء, عشرات, السنين]
1                                                                                        [ابشرك, ٣خطوط, نقلتها, وتوبه, وماعاد, رجعه]
2                                                                [اخذته, رقمي, الثاني, فرع, الظهران, مول, الموضوع, دقايق, شكرا, stc]
3                                                                                                                   [يعوضني, stc, 🥺]
4                                            [ترا, ماينفع, معهم, انك, تروح, لاقرب, فرع, وتفاهم, معهم, وقلهم, بحول, عنكم, شوف, يرجعو]
                                                                   ...                                                              
128                                                                                         [ياخي, ليه, ماتردون, وتخلصون, مشكلتي, 🤌]
129                                                                  

In [19]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

cleaned_tweets = file_model['clean_tweet'].apply(lambda x: " ".join(x))

# unigram = word_vectorizer.fit(file_model['clean_tweet'])
unigramdataGet= word_vectorizer.fit_transform(cleaned_tweets)
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

,10,13,5g,90,iphone,my,mystc,stc,stcpay,stcوش,...,١٨,٢٠,٢٠٣٠,٢٤,٢٩,٣خطوط,٣٠١٢,٤ساعات,٥٠,٨سنوات
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
w2v_model = Word2Vec(min_count=3,
                     window=3,
                     min_alpha=0.0007, 
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 05:37:57: collecting all words and their counts
INFO - 05:37:57: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 05:37:57: collected 833 word types from a corpus of 1141 raw words and 125 sentences
INFO - 05:37:57: Loading a fresh vocabulary
INFO - 05:37:57: effective_min_count=3 retains 61 unique words (7% of original 833, drops 772)
INFO - 05:37:57: effective_min_count=3 leaves 244 word corpus (21% of original 1141, drops 897)
INFO - 05:37:57: deleting the raw counts dictionary of 833 items
INFO - 05:37:57: sample=0.001 downsamples 61 most-common words
INFO - 05:37:57: downsampling leaves estimated 74 word corpus (30.4% of prior 244)
INFO - 05:37:57: estimated required memory for 61 words and 100 dimensions: 79300 bytes
INFO - 05:37:57: resetting layer weights


Time to build vocab: 0.0 mins


In [21]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 05:37:57: training model with 7 workers on 61 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
INFO - 05:37:57: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 5 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 4 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 3 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 2 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 1 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 0 more threads
INFO - 05:37:57: EPOCH - 1 : training on 1141 raw words (77 effective words) took 0.0s, 9111 effective words/s
INFO - 05:37:57: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 5 more threads
INFO - 05:37:57: worker thread finished; awaiting finish of 4 more threads
INFO - 0

INFO - 05:37:58: worker thread finished; awaiting finish of 0 more threads
INFO - 05:37:58: EPOCH - 13 : training on 1141 raw words (75 effective words) took 0.0s, 9103 effective words/s
INFO - 05:37:58: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 5 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 4 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 3 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 2 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 1 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 0 more threads
INFO - 05:37:58: EPOCH - 14 : training on 1141 raw words (75 effective words) took 0.0s, 9633 effective words/s
INFO - 05:37:58: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 5 more threads
INFO - 05:37:58: worker th

INFO - 05:37:58: worker thread finished; awaiting finish of 1 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 0 more threads
INFO - 05:37:58: EPOCH - 26 : training on 1141 raw words (69 effective words) took 0.0s, 5461 effective words/s
INFO - 05:37:58: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 5 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 4 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 3 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 2 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 1 more threads
INFO - 05:37:58: worker thread finished; awaiting finish of 0 more threads
INFO - 05:37:58: EPOCH - 27 : training on 1141 raw words (66 effective words) took 0.0s, 4737 effective words/s
INFO - 05:37:58: worker thread finished; awaiting finish of 6 more threads
INFO - 05:37:58: worker th

Time to train the model: 0.01 mins


In [24]:
# store the word2vec model
w2v_model.save("word2vec.model")

INFO - 05:38:31: saving Word2Vec object under word2vec.model, separately None
INFO - 05:38:31: not storing attribute vectors_norm
INFO - 05:38:31: not storing attribute cum_table
INFO - 05:38:31: saved word2vec.model


In [25]:
word_vectors = Word2Vec.load("word2vec.model").wv

INFO - 05:38:33: loading Word2Vec object from word2vec.model
INFO - 05:38:33: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 05:38:33: setting ignored attribute vectors_norm to None
INFO - 05:38:33: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 05:38:33: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 05:38:33: setting ignored attribute cum_table to None
INFO - 05:38:33: loaded word2vec.model


In [26]:
file_model.columns

Index(['ID', 'text', 'label', 'tweet_type', 'tweet_topic', 'clean_tweet'], dtype='object')

In [27]:
file_export = file_model.copy()
file_export['old_tweets'] = file_export.clean_tweet
file_export.old_tweets = file_export.old_tweets.str.join(' ')
file_export.clean_tweet = file_export.clean_tweet.apply(lambda x: ' '.join(bigram[x]))

In [28]:
file_export[['old_tweets','clean_tweet']]

,old_tweets,clean_tweet
0,ابشرك بتدخل اليوم العاشر عشان يردوا يخدمونك ياخي تحس stc مايدرون رءيه ٢٠٣٠ الحين ونفس الاخطاء عشرات السنين,ابشرك بتدخل اليوم العاشر عشان يردوا يخدمونك ياخي تحس stc مايدرون رءيه ٢٠٣٠ الحين ونفس الاخطاء عشرات السنين
1,ابشرك ٣خطوط نقلتها وتوبه وماعاد رجعه,ابشرك ٣خطوط نقلتها وتوبه وماعاد رجعه
2,اخذته رقمي الثاني فرع الظهران مول الموضوع دقايق شكرا stc,اخذته رقمي الثاني فرع الظهران مول الموضوع دقايق شكرا stc
3,يعوضني stc 🥺,يعوضني stc 🥺
4,ترا ماينفع معهم انك تروح لاقرب فرع وتفاهم معهم وقلهم بحول عنكم شوف يرجعو,ترا ماينفع معهم انك تروح لاقرب فرع وتفاهم معهم وقلهم بحول عنكم شوف يرجعو
...,...,...
128,ياخي ليه ماتردون وتخلصون مشكلتي 🤌,ياخي ليه ماتردون وتخلصون مشكلتي 🤌
129,خلاص فعلته بطريقه ثانيه mystc وحطيته بالرمز اليدوي,خلاص فعلته بطريقه ثانيه mystc وحطيته بالرمز اليدوي
130,الخير الغيت رقمي المفوتر ونزلت مديوانيه سلامات,الخير الغيت رقمي المفوتر ونزلت مديوانيه سلامات
131,سوا بوست بلس,سوا_بوست بلس


In [29]:
file_export.to_csv('cleaned_dataset for training.csv', index=False)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [45]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet = word_vectorizer.fit(file_export['clean_tweet']).transform(file_export['clean_tweet'])
# unigramdataGet= word_vectorizer.transform(unigram)
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

,10,13,5g,90,iphone,my,mystc,stc,stcpay,stcوش,...,١٧٠,١٨,٢٠,٢٠٣٠,٢٤,٢٩,٣خطوط,٣٠١٢,٤ساعات,٨سنوات
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
import pickle

In [32]:
vec_file = 'vectorizer.pickle'
pickle.dump(word_vectorizer, open(vec_file, 'wb'))

In [33]:
vec_file

'vectorizer.pickle'

In [34]:
pro= preprocessing.LabelEncoder()
encpro=pro.fit_transform(file_export['label'])
file_export['label'] = encpro

In [35]:
y=file_export['label']
X=unigramdata_features

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [37]:
X_train.head()

,10,13,5g,90,iphone,my,mystc,stc,stcpay,stcوش,...,١٨,٢٠,٢٠٣٠,٢٤,٢٩,٣خطوط,٣٠١٢,٤ساعات,٥٠,٨سنوات
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
nb=GaussianNB()
nb= nb.fit(X_train , y_train)
nb

GaussianNB()

In [39]:
y_pred = nb.predict(X_test)
nb_1=nb.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(nb.score(X_test, y_test)))
print('Accuracy= {:.3f}'.format(nb.score(X_train , y_train)))

Accuracy= 0.640
Accuracy= 0.960


In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.67      0.75        18
           1       0.57      0.67      0.62         6
           2       0.00      0.00      0.00         1

    accuracy                           0.64        25
   macro avg       0.48      0.44      0.46        25
weighted avg       0.75      0.64      0.69        25



In [50]:
import pickle

In [51]:
mod_file = 'classification.model'
pickle.dump(nb, open(mod_file, 'wb'))

In [41]:
replies = pd.read_excel('test_set.xlsx')

In [42]:
replies.shape

(133, 5)

In [43]:
replies.columns

Index(['ID', 'text', 'label', 'tweet_type', 'tweet_topic'], dtype='object')

In [44]:
replies['text']=replies['text'].apply(str)

In [45]:
replies['clean_tweet'] = replies['text'].apply(text_preprocessing)

In [46]:
replies.clean_tweet  = replies.clean_tweet.apply(lambda x: ' '.join(bigram[x]))


In [47]:
replies['clean_tweet'].shape

(133,)

In [48]:
loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))

In [52]:
loaded_model = pickle.load(open('classification.model', 'rb'))

In [53]:
pickle.load(open('vectorizer.pickle', 'rb'))

TfidfVectorizer(max_features=10000, strip_accents='unicode', sublinear_tf=True)

In [54]:
loaded_vectorizer.transform('كلمني انستا')

ValueError: Iterable over raw text documents expected, string object received.

In [64]:
type(replies['clean_tweet'][0])

list

In [59]:
replies['label'] = loaded_model.predict(loaded_vectorizer.transform(replies['clean_tweet']))

AttributeError: 'list' object has no attribute 'lower'

In [93]:
loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))

In [94]:
loaded_vectorizer.transform(replies['clean_tweet'])

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
 loaded_model.predict(loaded_vectorizer(replies['clean_tweet'])

In [82]:
for i in replies:
    replies['label']=replies.clean_tweet.apply(classify_utterance([i]))
    

NotFittedError: The TF-IDF vectorizer is not fitted

In [ ]:
unigramdataGet= word_vectorizer.fit_transform(file_export['clean_tweet'].astype('str'))
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1


In [66]:
# for i in replies:
#     replies['label']=replies.clean_tweet.apply(classify_utterance([i]))

NotFittedError: The TF-IDF vectorizer is not fitted

In [49]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet= word_vectorizer.transform(replies['clean_tweet'].to_list())
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

NotFittedError: The TF-IDF vectorizer is not fitted

In [ ]:
replies['label']=replies['clean_tweet'].apply(nb.predict)

In [ ]:
RC= RidgeClassifier()
RC= RC.fit(X_train , y_train)
RC

In [ ]:
y_pred = RC.predict(X_test)
rc_1=RC.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(RC.score(X_test, y_test)))

In [ ]:
PC= PassiveAggressiveClassifier()
PC= PC.fit(X_train , y_train)
PC

In [ ]:
y_pred = PC.predict(X_test)
pc_1=PC.score(X_test, y_test)
pc_2=PC.score(X_train , y_train)
print('Accuracy= {:.3f}'.format(PC.score(X_test, y_test)))


In [ ]:

LR= LogisticRegression(penalty = 'l2', C = 1)
LR= LR.fit(X_train , y_train)
LR



In [ ]:
y_pred = LR.predict(X_test)
lr_1=LR.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(LR.score(X_test, y_test)))

In [ ]:
from prettytable import PrettyTable

In [ ]:
x = PrettyTable()
print('\n')
print("Comparison of all algorithms on F1 score")
x.field_names = ["Model", "Accuracy"]


x.add_row(["Naive Bayes Algorithm", round(nb_1,2)])
x.add_row(["Ridge Classifier Algorithm",  round(rc_1,2)])
x.add_row(["Passive Aggressive Classifier Algorithm", round(pc_1,2)])
x.add_row(["Logistic Regression Algorithm", round(lr_1,2)])

print(x)
print('\n')

In [ ]:
X.shape

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
stc=["الخدمة جيدة"]

In [ ]:
x = word_vectorizer.transform(stc)

In [ ]:
x = x.toarray()

In [ ]:
pred=nb.predict(x)
pred=pro.inverse_transform(pred)
prediction=pd.DataFrame(pred, columns=['Prediction']) 
print (prediction)

In [ ]:
tweets = file_export.copy()

In [ ]:
X_train.shape

In [ ]:
tweets.clean_tweet

In [ ]:
tweets.columns

In [ ]:
unigramdataGet= word_vectorizer.fit_transform(file_export['clean_tweet'].astype('str'))
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

In [ ]:
X.shape

In [ ]:
nb.predict(X)

In [ ]:
tweets["predict"] = nb.predict(tweets["clean_tweet"])

In [ ]:
prediction = []

In [ ]:
for i in tweets:
  X = word_vectorizer.transform(i)
  pred=pro.inverse_transform(pred)
  prediction=pd.DataFrame(pred, columns=['Prediction']) 
  print (prediction)

In [ ]:
file_export

In [ ]:
file_export = reset.index()